In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import os
import matplotlib.pyplot as plt
import joblib
import json

# Load the original dataset
df = pd.read_csv('Electric_vehicle_population_By_country.csv')

# Convert date column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])
df['County'] = df['County'].str.strip()
df['State'] = df['State'].str.strip()

# Filter for Washington state and passenger vehicles, as in the original project
df = df[(df['State'] == 'WA') & (df['Vehicle Primary Use'] == 'Passenger')].copy()

# Convert 'EV Total' to numeric, handling commas
df['Electric Vehicle (EV) Total'] = pd.to_numeric(df['Electric Vehicle (EV) Total'].astype(str).str.replace(',', ''), errors='coerce')

# Aggregate to get total monthly EV count for the entire state
monthly_df = df.groupby('Date')['Electric Vehicle (EV) Total'].sum().reset_index()

# Sort by date
monthly_df.sort_values(by='Date', inplace=True)

# Create a cumulative count for visualization and modeling
monthly_df['Cumulative_EV'] = monthly_df['Electric Vehicle (EV) Total'].cumsum()

# Rescale the data for the LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(monthly_df['Cumulative_EV'].values.reshape(-1, 1))

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Use a look-back window of 12 months for the LSTM
look_back = 12
X, y = create_dataset(scaled_data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split data into train and test sets (90% train, 10% test)
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(look_back, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)

# Make predictions on test data
train_predict = model.predict(X_train, verbose=0)
test_predict = model.predict(X_test, verbose=0)

# Invert predictions to original scale
train_predict_inv = scaler.inverse_transform(train_predict)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
test_predict_inv = scaler.inverse_transform(test_predict)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE and R-squared
train_score = np.sqrt(mean_squared_error(y_train_inv, train_predict_inv))
test_score = np.sqrt(mean_squared_error(y_test_inv, test_predict_inv))
r2_score_val = r2_score(y_test_inv, test_predict_inv)

print(f'Train RMSE: {train_score:.2f}')
print(f'Test RMSE: {test_score:.2f}')
print(f'R-squared: {r2_score_val:.4f}')

# Save the model and scaler
model.save('ev_lstm_model.keras')
joblib.dump(scaler, 'ev_scaler.pkl')

# Save accuracy metrics to a JSON file
metrics = {
    "test_rmse": test_score,
    "r2_score": r2_score_val
}
with open('metrics_dl.json', 'w') as f:
    json.dump(metrics, f)



Epoch 1/100


c:\Users\aravi\OneDrive\Desktop\ev\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


58/58 - 5s - 85ms/step - loss: 0.0068
Epoch 2/100
58/58 - 1s - 9ms/step - loss: 0.0017
Epoch 3/100
58/58 - 1s - 15ms/step - loss: 0.0029
Epoch 4/100
58/58 - 1s - 13ms/step - loss: 0.0023
Epoch 5/100
58/58 - 1s - 14ms/step - loss: 0.0012
Epoch 6/100
58/58 - 1s - 13ms/step - loss: 0.0021
Epoch 7/100
58/58 - 1s - 13ms/step - loss: 0.0023
Epoch 8/100
58/58 - 1s - 12ms/step - loss: 0.0012
Epoch 9/100
58/58 - 1s - 13ms/step - loss: 0.0013
Epoch 10/100
58/58 - 1s - 13ms/step - loss: 0.0013
Epoch 11/100
58/58 - 1s - 14ms/step - loss: 7.6517e-04
Epoch 12/100
58/58 - 1s - 14ms/step - loss: 0.0016
Epoch 13/100
58/58 - 1s - 13ms/step - loss: 0.0023
Epoch 14/100
58/58 - 1s - 13ms/step - loss: 0.0015
Epoch 15/100
58/58 - 1s - 12ms/step - loss: 0.0014
Epoch 16/100
58/58 - 1s - 13ms/step - loss: 9.6162e-04
Epoch 17/100
58/58 - 1s - 13ms/step - loss: 7.9771e-04
Epoch 18/100
58/58 - 1s - 11ms/step - loss: 0.0010
Epoch 19/100
58/58 - 1s - 13ms/step - loss: 0.0014
Epoch 20/100
58/58 - 1s - 12ms/step - los

NameError: name 'json' is not defined